In [1]:
import Pkg
include("EVGONN.jl")
using .EVGONN
using RDatasets
using StatsBase

In [2]:
iris = dataset("datasets", "iris");
X = Array(iris[1:4]);
y = iris[5];
n = length(y);

In [3]:
dt = fit(ZScoreTransform, X, dims=1)
X = StatsBase.transform(dt, X)
X = reshape(X, (4, 1, n));

In [4]:
function get_indices(v)
    return CategoricalArrays.order(v.pool)[v.refs]
end
    
refs=get_indices(y);
y=convert(Vector{Int},refs);

In [5]:
y_mod = zeros(UInt8, (n, 3))
for i in 1:n
    y_mod[i, y[i]] = 0x01
end

In [30]:
nn = NeuralNetwork(4, (20,8), 3, 3, η=0.01, β1=0.01, β2=0.0000000001);

In [31]:
function training(iters=10000)
    costs = zeros(n);
    foo(x) = min(0.0000000005x^2 + 0.01, 0.51)
    for it in 1:iters
        for i in 1:n
            result = train(prepare(X[:, 1, i]'), prepare(y_mod[i, :]'), nn)
            costs[i] = result["cost"]
        end
        #nn.learning_rate = foo(it)
        if it % 100 == 0
            #nn.β2 = nn.β2 * 0.75
            println("error = ", sum(costs))
        end
    end
end

training (generic function with 2 methods)

In [32]:
training()

error = 39.4556188018448
error = 40.06760531812704
error = 39.31345562781418
error = 38.38964851953039
error = 37.422589942147205
error = 36.36576486833826
error = 35.14097273371367
error = 33.52760492016711
error = 31.764780470596598
error = 30.09906042707304
error = 28.654612595715083
error = 27.355562144090804
error = 26.169951299288623
error = 25.10439303659338
error = 24.153771358172637
error = 23.304925942516203
error = 22.5568437879672
error = 21.900684284340883
error = 21.286438882020473
error = 20.634421999032003
error = 19.930583854220153
error = 19.123824789373568
error = 18.308775329957562
error = 17.532357899735004
error = 16.82941084902484
error = 16.21074366098819
error = 15.644840645960306
error = 15.120099434150214
error = 14.638943116368955
error = 14.201376660343936
error = 13.790698048590349
error = 13.388281713875875
error = 13.025371776314124
error = 12.708819789028723
error = 12.380822319870125
error = 12.124264392589984
error = 11.898024183168877
error = 11.7086

In [33]:
function check()
    checks = Dict("HIT" => 0, "MISS" => 0)
    for i in 1:n
        #println(predict(prepare(X[:, 1, i]'), nn)["result"])
        #println(argmax(predict(prepare(X[:, 1, i]'), nn)["result"])[2])
        #println(y[i])
        if argmax(EVGONN.predict(prepare(X[:, 1, i]'), nn)["result"])[2] == y[i]
            checks["HIT"] += 1
        else
            checks["MISS"] += 1
        end
    end
    println("accuracy = $(checks["HIT"] / 150)%")
end

check (generic function with 1 method)

In [34]:
check()

accuracy = 0.9333333333333333%
